In [1]:
# Initialization
from assets.pdfshow import *
from assets.startup import *

### README
> **请首先尝试查看 `ex1.ipynb`, 此为备用方案！**
>
> 这是 `ex1.ipynb` 的精简版本，作为 `ex1.ipynb` 实在无法运行时的补救方案；删去了 PDF 嵌入，故请结合 `latex/` 目录下的 `print.pdf` 进行阅读。

前文见 <a href="latex/print.pdf">`latex/print.pdf`</a>. 续：

### (c) $\ln\det H_n$ 的估计
下面对 $n \le 10$ 验证 $\det H_n$ 的估计值。首先使用 `sympy` 构造精确的 Hilbert 矩阵：

In [2]:
import sympy as sym
sym.init_printing(wrap_line=False, pretty_print=False)

In [3]:
def mrange(a, b):
    """ Sensible range """
    return range(a, b + 1)
def hilbertMatrix(n: int):
    """ Construct Hilbert matrix, using exact number """
    return sym.Matrix(
        [[sym.Integer(1) / sym.Integer(i + j - 1)
          for i in mrange(1, n)]
         for j in mrange(1, n)])

相应地，计算行列式的准确值如下：

In [4]:
hilbertDetExact = [hilbertMatrix(n).det() for n in mrange(1, 10)]
hilbertDetExact

[1, 1/12, 1/2160, 1/6048000, 1/266716800000, 1/186313420339200000, 1/2067909047925770649600000, 1/365356847125734485878112256000000, 1/1028781784378569697887052962909388800000000, 1/46206893947914691316295628839036278726983680000000000]

上述操作是为了 *保证* 参考数据的有效性；由此得到参考数据列表（对数值）：

In [5]:
from math import *
hilbertDetLogRef = [log(item) for item in hilbertDetExact]
hilbertDetLogRef

[0.0, -2.4849066497880004, -7.67786350067821, -15.615238196841506, -26.309453258276445, -39.76620670609766, -55.98858020684256, -74.97842732916048, -96.73694927439705, -121.2649687493037]

相应地，由对数和阶乘表达的公式在理论上是精确的，但计算过程中 *可能* 存在舍入误差；结果如下：

In [6]:
def logC(n: int) -> float:
    """ Calculate log c_n; log = ln, i.e. base e """
    return sum([log(factorial(m)) for m in mrange(1, n - 1)])

hilbertDetLogTheoretical = [4 * logC(n) - logC(2 * n) for n in mrange(1, 10)]
hilbertDetLogTheoretical

[0.0, -2.4849066497880004, -7.67786350067821, -15.615238196841508, -26.309453258276445, -39.76620670609766, -55.988580206842556, -74.97842732916047, -96.736949274397, -121.26496874930368]

可见还好，$n\le 10$时的数值误差并不显著。相应地，还有渐进近似：

In [7]:
# Rough asymptotic
hilbertDetLogAsymptotic = [log(4 ** (- n ** 2)) for n in mrange(1, 10)]
hilbertDetLogAsymptotic

[-1.3862943611198906, -5.545177444479562, -12.476649250079015, -22.18070977791825, -34.657359027997266, -49.90659700031606, -67.92842369487464, -88.722839111673, -112.28984325071114, -138.62943611198907]

In [8]:
def integralApprox(k: int) -> float:
    """ Estimate the sum of m*log(m), by integration """
    return 1 / 8 * (4 - 9 * log(3) - 2 * k * (1 + k) * (1 + log(4))
                    + log(256) + (1 + 2 * k)**2 * log(1 + 2 * k))
def hilbertDetLogApproxFunction(n: int) -> float:
    """ Better asymptotic: Stirling approximation of det(H_n) """
    return 4 * integralApprox(n - 1) - integralApprox(2 * n - 1) \
        + (2 * n - 1) * log(n - 1) - (n - 1 / 4) * log(2 * n - 1) \
        + (n - 9 / 4) * log(2 * pi) + 3 / 2
# Nice asymptotic
hilbertDetLogApprox = ['N/A'] + [hilbertDetLogApproxFunction(n) for n in mrange(2, 10)]
hilbertDetLogApprox

['N/A',
 -5.599065707183972,
 -10.780936043131339,
 -18.735491636544033,
 -29.448858142186378,
 -42.923424656009416,
 -59.16187737525633,
 -78.16620649373371,
 -99.93783436065031,
 -124.47779117064434]

各种方法得到的 $\ln\det H_n$ 估计值与精确值综合如下：

In [9]:
import pandas as pd
def transpose(l: list) -> list:
    return [[l[j][i] for j in range(len(l))] for i in range(len(l[0]))]
hilbertDetLogData = pd.DataFrame(
    transpose([hilbertDetLogRef, hilbertDetLogTheoretical,
               hilbertDetLogApprox, hilbertDetLogAsymptotic]),
    columns=['Exact', 'Theoretical', 'Nice asymptotic', 'Rough asymptotic'])
hilbertDetLogData.index = list(mrange(1, 10))
hilbertDetLogData

,Exact,Theoretical,Nice asymptotic,Rough asymptotic
1,0.000000,0.000000,N/A,-1.386294
2,-2.484907,-2.484907,-5.59907,-5.545177
3,-7.677864,-7.677864,-10.7809,-12.476649
4,-15.615238,-15.615238,-18.7355,-22.180710
5,-26.309453,-26.309453,-29.4489,-34.657359
6,-39.766207,-39.766207,-42.9234,-49.906597
7,-55.988580,-55.988580,-59.1619,-67.928424
8,-74.978427,-74.978427,-78.1662,-88.722839
9,-96.736949,-96.736949,-99.9378,-112.289843
10,-121.264969,-121.264969,-124.478,-138.629436


相对精确值的偏差（从 $n = 2$ 起）：

In [10]:
def relativeError(aPlus: float, a: float) -> float:
    return (aPlus - a) / a

hilbertDetLogError = pd.DataFrame(
    [[relativeError(hilbertDetLogTheoretical[n], hilbertDetLogRef[n]),
      relativeError(hilbertDetLogApprox[n], hilbertDetLogRef[n]),
     relativeError(hilbertDetLogAsymptotic[n], hilbertDetLogRef[n])]
     for n in range(0 + 1, 10)],
    columns=['Theoretical', 'Nice asymptotic', 'Rough asymptotic'])
hilbertDetLogError.index = list(mrange(2, 10))
hilbertDetLogError

,Theoretical,Nice asymptotic,Rough asymptotic
2,-0.000000e+00,1.253230,1.231544
3,-0.000000e+00,0.404158,0.625016
4,1.137579e-16,0.199821,0.420453
5,-0.000000e+00,0.119326,0.317297
6,-0.000000e+00,0.079394,0.255000
7,-1.269085e-16,0.056678,0.213255
8,-1.895326e-16,0.042516,0.183312
9,-4.407061e-16,0.033089,0.160775
10,-2.343769e-16,0.026494,0.143194


可见，采用渐进表达式给出的 $\ln\det H_n$ 估计值在 $n$ 充分大时是比较有效的。事实上，不妨考察更大的 $n$ 值：

In [11]:
largeNs = [100, 1000, 10000]
# WARNING: This will take a loooong time! Don't run unless prepared! 
# hilbertDetLogTheoretical_largeNs = {
#     f'{n}': 4 * logC(n) - logC(2 * n) for n in largeNs
# }
hilbertDetLogTheoretical_largeNs = {
    '100': -13680.745699832958,
    '1000': -1384458.6494934857,
    '10000': -138611060.08241153
}
hilbertDetLogTheoretical_largeNs

{'100': -13680.745699832958,
 '1000': -1384458.6494934857,
 '10000': -138611060.08241153}

此时对数 + 阶乘的精确公式需要巨大的计算量，不切实际；而渐进表达式的精度随 $n$ 增大进一步提高，如下所示：

In [12]:
hilbertDetLogError_largeN = pd.DataFrame(
    [[relativeError(hilbertDetLogApproxFunction(n),
                    hilbertDetLogTheoretical_largeNs[f'{n}']),
      relativeError(- n**2 * log(4),
                    hilbertDetLogTheoretical_largeNs[f'{n}'])]
     for n in largeNs],
    columns=['Nice asymptotic', 'Rough asymptotic'])
hilbertDetLogError_largeN.index = largeNs
hilbertDetLogError_largeN

,Nice asymptotic,Rough asymptotic
100,2.552795e-04,0.013318
1000,2.729996e-06,0.001326
10000,2.934343e-08,0.000133


即此时应当采用渐进展开来估计 $\ln\det H_n$. 
> 事实上，还有另一种办法，即直接给出 $\ln\det H_n$ 的 *解析* 表达式；这竟然是可以做到的！伟大的 Mathematica 告诉我们：
> $$ \ln\det H_n = 4 \ln G(x+1) - \ln G(2x+1) $$
> 其中 $G$ 为 Barnes G 函数（Mathematica `BarnesG`）。在此基础上便可给出 $\ln\det H_n$ 任意精度的值。

### (d) $H_n \cdot c = b$ 的求解

对 $b = (1,1,\cdots,1)$, 求解上述线性方程，由此探究 $H_n$ 的近奇异性。注意到 $n = 1$ 的情形是平凡的，因此我们只看 $n = 2,3,\dots,11$. <br/>
（为补偿去掉的 $n = 1$, 增加了一个数据点 $n = 11$. ）

类似前文，首先构造精确的参考解：

In [13]:
def hilbertSol(n: int):
    """ Calculate H.inverse dot b, b: constant vector of 1 """
    return list(
        hilbertMatrix(n).inv() * sym.Matrix(eval('[1] * n'))
    )
hilbertSolList = [hilbertSol(n) for n in mrange(2, 11)]
hilbertSolList

[[-2, 6], [3, -24, 30], [-4, 60, -180, 140], [5, -120, 630, -1120, 630], [-6, 210, -1680, 5040, -6300, 2772], [7, -336, 3780, -16800, 34650, -33264, 12012], [-8, 504, -7560, 46200, -138600, 216216, -168168, 51480], [9, -720, 13860, -110880, 450450, -1009008, 1261260, -823680, 218790], [-10, 990, -23760, 240240, -1261260, 3783780, -6726720, 7001280, -3938220, 923780], [11, -1320, 38610, -480480, 3153150, -12108096, 28588560, -42007680, 37413090, -18475600, 3879876]]

下面编写通用的方程求解代码；首先有适用范围最广的高斯消元法（Gaussian elimination, GEM, 虽说按道理来说应当叫九章算法）：

In [14]:
def gem(m: list, b: list) -> list:
    """ Gaussian Elimination: m x = b;
        :param m: real matrix as (column_dim) * (row_dim) list;
        :param b: row_dim-vector as list;
        :return:  solution as list; return blank list if fails.
    """

    # Define dimension:
    column_dim = len(b)  # Must have: len(m) == len(b).
    row_dim = len(m[0])  # m: NOT necessarily a square.
    try:
        augmented_m = [ m[i][:] + [b[i]] for i in range(column_dim) ]
    except IndexError:
        print('GEM Error：dimension inconsistent!')  # When len(m) != len(b).

    # Gaussian elimination -> upper triangular matrix
    # Note: row has row_dim, but its element is labeled by column_index.
    #       column has column_dim, but its element is labeled by row_index.
    for column_index in range(row_dim - 1):      # -1: no need for last column.
        pivot_row = column_index                 # Start from diagonal element
        pivot_value = augmented_m[pivot_row][column_index]
        for row in range(column_index + 1, column_dim):   # Partial pivoting
            if abs(augmented_m[row][column_index]) > abs(pivot_value):
                pivot_row = row
                pivot_value = augmented_m[row][column_index]
        if pivot_row != column_index:                     # Row switch
            augmented_m[pivot_row], augmented_m[column_index] \
                = augmented_m[column_index], augmented_m[pivot_row]
        for row in range(column_index + 1, column_dim):
            try:                                          # Gaussian Elimination
                resize_factor = augmented_m[row][column_index] / pivot_value
            except ZeroDivisionError:
                print("GEM Error：matrix is singular!")
                return []
            for column in range(column_index, row_dim + 1):
                augmented_m[row][column] \
                    -= augmented_m[column_index][column] * resize_factor

    # Substitution:
    x = []
    for index in range(column_dim - 1, -1, -1):
        partial_mx = sum(
            [ x[j] * augmented_m[index][row_dim - j - 1]
              for j in range(len(x)) ]   # Partial sum of m * solved_xComponent
        )                                # x[] is reversed for convenience
        new_xComponent = (augmented_m[index][row_dim] - partial_mx) \
            / augmented_m[index][index]
        x.append(new_xComponent)
    x.reverse()                          # Reverse back.
    return x

然后是 Cholesky 分解：

In [15]:
def cholesky(m: list, b: list) -> list:
    """ Cholesky Decomposition: m x = b;
        :param m: real *positive-definite* matrix, as dim * dim (square) list;
        :param b: dim-vector as list;
        :return:  solution as list; return blank list if fails.
    """

    # Note: m must be a square matrix!
    dim = len(m)
    if dim != len(b):
        print('Cholesky Error：dimension inconsistent!')
        return []

    # Cholesky decomposition
    h = [[0. for column in range(dim)] for row in range(dim)]
    h[0][0] = sqrt(m[0][0])
    for i in range(1, dim):          # Hard to ensure positive-definiteness;
        try:                         # therefore prepared for error!
            for j in range(i):
                h[i][j] = (
                    m[i][j]
                    - sum([ h[i][k] * h[j][k] for k in range(j) ])
                ) / h[j][j]
            hii_squared = m[i][i] - sum([ h[i][k] ** 2 for k in range(i) ])
            h[i][i] = sqrt(hii_squared)
        except (ValueError, ZeroDivisionError):
            print(f'Cholesky Error: h[{i:d}][{i:d}]^2 = {hii_squared:g} <= 0, '
                  'matrix ill-conditioned!')
            return []

    # Substitution:
    h_t = transpose(h)     # Nice to have real matrix! adjoint == transpose
    y = []
    for index in range(dim):
        new_yComponent = (
            b[index]
            - sum([ y[j] * h[index][j] for j in range(len(y)) ])
        ) / h[index][index]
        y.append(new_yComponent)
    x = []
    for index in range(dim - 1, -1, -1):
        new_xComponent = (
            y[index]
            - sum([ x[j] * h_t[index][dim - j - 1] for j in range(len(x)) ])
        ) / h_t[index][index]
        x.append(new_xComponent)
    x.reverse()
    return x

重定义 `float` 类型的 $H_n$: 

In [16]:
def hilbertMatrixNumerical(n):
    return [ [ 1. / (i + j - 1)
               for i in mrange(1, n)]
             for j in mrange(1, n)]

以偏离最大的分量为准，比较相对误差如下：

In [17]:
# Create error list: index 0 for GEM, 1 for Cholesky.
errorList = [[0. for n in mrange(2, 11)] for method in range(2)]
for n in mrange(2, 11):
    m = hilbertMatrixNumerical(n)
    b = [1.] * n
    x_gem = gem(m, b)
    x_cholesky = cholesky(m, b)
    # Index handling is terrifying! mrange(2, 11) -> 0, 1, ... , 8
    errorList[0][n - 2] \
        = max([ relativeError(x_gem[i], hilbertSolList[n - 2][i])
                for i in range(n)])
    errorList[1][n - 2] \
        = max([ relativeError(x_cholesky[i], hilbertSolList[n - 2][i])
                for i in range(n)])
errorTable = pd.DataFrame(transpose(errorList), columns=['GEM', 'Cholesky'])
errorTable.index = mrange(2, 11)
errorTable.to_csv('methodsCompare.csv')
errorTable

,GEM,Cholesky
2,4.44089209850063e-16,2.22044604925031e-16
3,6.51330841113425e-15,4.73695157173400e-15
4,-8.72952503933837e-14,-8.66862137627322e-14
5,3.66843063862429e-13,-2.51807322788481e-12
6,1.86702209248324e-10,-1.79028060614141e-12
7,6.14563029038046e-9,1.35239426007112e-9
8,-1.98951780863168e-8,-4.47140785234744e-8
9,-2.87910455529576e-6,-3.98234800962092e-6
10,-8.32347897425841e-5,-0.000157734346871478
11,-0.00201528738811147,-0.00378353196020243


图像如下，注意纵轴为对数标度！
> 插图见：<a href="latex/sections/3.3.errorplot.pdf">`latex/sections/3.3.errorplot.pdf`</a>

相对误差随阶数 $n$ 增大而近乎指数地增长，可见这一问题确实是非常地病态。

$n$ 较小时，两种算法在精确程度方面各有胜负；但当 $8 \le n \le 11$ 时 Cholesky 分解给出的误差稍大。原因在于，Cholesky 分解过程中必须除以对角元 `h[j][j]`, 而 `h[j][j]` 将随 $n$ 增大而迅速趋于零（行为与 $\det H_n$ 相近）；不得不除以一个近零的数，这显著地影响了 Cholesky 分解的精度。与之相比，GEM 通过支点遴选 *部分地* 避免了这一问题，因此在大 $n$ 时有相对较好的表现。